### Long Context Handling Capability

This notebook illustrates how you can use `capabilities` to give any `Conversable` agent an ability to handle long contexts.

In [1]:
## Uncomment to install pyautogen if you don't have it already
#! pip install pyautogen

In [2]:
import autogen
from autogen.agentchat.contrib.capabilities import context_handling

In [3]:
config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list,
    },
)
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=10,
)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10")

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot and save a graph of the function \( x^2 \) from \( -10 \) to \( 10 \), we need to use a library like `matplotlib`. If you don't have `matplotlib` installed, you will need to install it first using `pip`. 

Here’s the plan we will follow:
1. Import the required libraries, which are `matplotlib` for plotting and `numpy` for generating the range of x-values.
2. Generate a range of x-values from -10 to 10.
3. Calculate the corresponding y-values for \( x^2 \).
4. Plot the graph using `matplotlib`.
5. Save the graph to a file, for example, as `"x_squared_plot.png"`.

I will give you the complete code to execute, step by step:

```python
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np

# Generate a range of x-values
x_values = np.linspace(-10, 10, 400)  # 400 points for smo